In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.special import legendre
from scipy.stats import unitary_group
from numpy import kron
from logger import Logger

In [4]:
class Grape(object):
    """A class for pulse-based simulation and optimization
    Args:
        taylor_terms: number of taylor expansion terms.
    """
    def __init__(self, taylor_terms=5, n_step=100, n_epoch=200, lr=2e-2):
        args = locals()
        self.taylor_terms = taylor_terms
        self.n_step = n_step
        self.log_dir = "./logs/"
        self.log_name = 'grape'
        self.n_epoch = n_epoch
        self.lr = lr

        self.logger = Logger()
        self.logger.write_text("arguments ========")
        for k, v in args.items():
            if k == 'self':
                continue
            self.logger.write_text("{}: {}".format(k, v))

    def train_fidelity_ibm1(self, init_u, H0, Hs, initial_states, target_states):
        """Control the systems to reach target states.
        Args:
            init_u: initial pulse.
            H0: a Hermitian matrix.
            Hs: a list of Hermitian matrics.
            initial_states: initial_states.
            target_states: target_states.
            dt: size of time step.
        Returns:
            us: optimized pulses.
        """
        self.logger.write_text("!!!! train_fidelity ========")

        lr = self.lr
        w_l2 = 0
        
        H0 = torch.tensor(self.c_to_r_mat(-1.j * self.dt * H0))
        Hs = [torch.tensor(self.c_to_r_mat(-1.j * self.dt * hs)) for hs in Hs]
        us = torch.tensor(init_u, requires_grad=True)
        optimizer = torch.optim.Adam([us], lr=lr)
        initial_states = torch.tensor(np.array(initial_states)).double().transpose(1, 0)
        target_states = torch.tensor(np.array(target_states)).double().transpose(1, 0)
        #sgm = torch.nn.Sigmoid()
        self.losses_energy = []
        for epoch in range(self.n_epoch):
            st = "params: {}".format(us)
            self.logger.write_text_aux(st)
            final_states = self.forward_simulate_ibm1(us, H0, Hs, initial_states)
            loss_fidelity = 1 - self.get_inner_product_2D(final_states, target_states)
            loss_l2 = torch.sqrt((us**2).mean())
            loss = loss_fidelity + loss_l2 * w_l2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            st = "epoch: {:04d}, loss: {:.16f}, loss_fid: {:.16f}".format(
                epoch, 
                float(loss.detach().numpy()), 
                float(loss_fidelity.detach().numpy()))
            self.logger.write_text(st)

            self.losses_energy.append(float(loss_fidelity.detach().numpy()))
        
        return us
        
    
    
    def train_fidelity_ibm2(self, init_u, H0, Hs, initial_states, target_states):
        """Control the systems to reach target states.
        Args:
            init_u: initial pulse.
            H0: a list of Hermitian matrices.
            Hs: a list of Hermitian matrices.
            initial_states: initial_states.
            target_states: target_states.
            dt: size of time step.
        Returns:
            us: optimized pulses.
        """
        self.logger.write_text("!!!! train_fidelity ========")

        lr = self.lr
        w_l2 = 0
        #max_amplitude = torch.from_numpy(np.ones(len(Hs)))
        H0 = [torch.tensor(self.c_to_r_mat(-1.j * self.dt * h0)) for h0 in H0]
        Hs = [torch.tensor(self.c_to_r_mat(-1.j * self.dt * hs)) for hs in Hs]
        us = torch.tensor(init_u, requires_grad=True)
        optimizer = torch.optim.Adam([us], lr=lr)
        initial_states = torch.tensor(np.array(initial_states)).double().transpose(1, 0)
        target_states = torch.tensor(np.array(target_states)).double().transpose(1, 0)

        self.losses_energy = []
        for epoch in range(self.n_epoch):
            st = "params: {}".format(us)
            self.logger.write_text_aux(st)
            
            final_states = self.forward_simulate_ibm2(us, H0, Hs, initial_states)
            loss_fidelity = 1 - self.get_inner_product_2D(final_states, target_states)
            loss_l2 = torch.sqrt((us**2).mean())
            loss = loss_fidelity + loss_l2 * w_l2
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            st = "epoch: {:04d}, loss: {:.16f}, loss_fid: {:.16f}".format(
                epoch, 
                float(loss.detach().numpy()), 
                float(loss_fidelity.detach().numpy()))
            self.logger.write_text(st)

            self.losses_energy.append(float(loss_fidelity.detach().numpy()))
        
        return us

    
    def forward_simulate_ibm1(self, us, H0, Hs, initial_states):
        """Forward time evolution.
        Args:
            us: pulses.
            H0: a Hermitian matrix.
            Hs: a list of Hermitian matrics.
            initial_states: initial states.
        Returns:
            final_states: final states.
        """
        #self.intermediate_states = []
        
        n_step = us.shape[0]
        n_param = us.shape[1]
        
        if len(Hs) != 2:
            print("Hs must have 2 elements!")
            
        if n_param != 2:
            print("Num. of parameters must be 2!")

        final_states = initial_states
        sgm = torch.nn.Sigmoid()
        
        for j in range(n_step):
            Hjdt = H0.clone()
            N = torch.sqrt(torch.sum(torch.square(us[j])))
            Hjdt += (2*sgm(N)-1) / N * (us[j][0] * Hs[0].clone() + us[j][1] * Hs[1].clone())
            final_states = torch.matmul(torch.matrix_exp(Hjdt), final_states)

        return final_states
    
    
    def forward_simulate_ibm2(self, us, H0, Hs, initial_states):
        """Forward time evolution.
        Args:
            us: pulses.
            H0: a Hermitian matrix.
            Hs: a list of Hermitian matrics.
            initial_states: initial states.
        Returns:
            final_states: final states.
        """
        #self.intermediate_states = []

        n_step = us.shape[0]
        n_param = us.shape[1]
        
        if len(H0) != 3:
            print("H0 must have 3 elements!")
        if len(Hs) != 4:
            print("Hs must have 4 elements!")
            
        if n_param != 8:
            print("Num. of parameters must be 8!")

        final_states = initial_states
        sgm = torch.nn.Sigmoid()

        for j in range(n_step):
            t = j * self.dt
            Nd0 = torch.sqrt(torch.sum(torch.square(us[j][0:2])))
            Nu0 = torch.sqrt(torch.sum(torch.square(us[j][2:4])))
            Nd1 = torch.sqrt(torch.sum(torch.square(us[j][4:6])))
            Nu1 = torch.sqrt(torch.sum(torch.square(us[j][6:8])))
            Hjdt = H0[0].clone() + np.cos(self.delta*t) * H0[1].clone() + np.sin(self.delta*t) * H0[2].clone()
            Hjdt += ((2*sgm(Nd0)-1) / Nd0 * us[j][0]  + (2*sgm(Nu0)-1) / Nu0 *
                     (np.cos(self.delta*t)*us[j][2] - np.sin(self.delta*t)*us[j][3])) * Hs[0].clone()
            Hjdt += ((2*sgm(Nd0)-1) / Nd0 * us[j][1] + (2*sgm(Nu0)-1) / Nu0 *
                     (np.sin(self.delta*t)*us[j][2] + np.cos(self.delta*t)*us[j][3])) * Hs[1].clone()
            Hjdt += ((2*sgm(Nd1)-1) / Nd1 * us[j][4] + (2*sgm(Nu1)-1) / Nu1 *
                     (np.cos(self.delta*t)*us[j][6] - np.sin(self.delta*t)*us[j][7])) * Hs[2].clone()
            Hjdt += ((2*sgm(Nd1)-1) / Nd1 * us[j][5] + (2*sgm(Nu1)-1) / Nu1 *
                     (np.sin(self.delta*t)*us[j][6] + np.cos(self.delta*t)*us[j][7])) * Hs[3].clone()
            final_states = torch.matmul(torch.matrix_exp(Hjdt), final_states)

        return final_states

    def save_plot(self, plot_name, us):
        return
    

    def get_inner_product_2D(self,psi1,psi2):
        """compute the inner product |psi1.dag() * psi2|.
        Args:
            psi1: a state.
            psi2: a psi2.
        Returns:
            norm: norm of the complex number.
        """
        state_dim = int(psi1.shape[0] / 2)
        psi_1_real = psi1[0:state_dim,:]
        psi_1_imag = psi1[state_dim:2*state_dim,:]
        psi_2_real = psi2[0:state_dim,:]
        psi_2_imag = psi2[state_dim:2*state_dim,:]

        ac = (psi_1_real * psi_2_real).sum(0) 
        bd = (psi_1_imag * psi_2_imag).sum(0) 
        bc = (psi_1_imag * psi_2_real).sum(0) 
        ad = (psi_1_real * psi_2_imag).sum(0) 
        reals = (ac + bd).sum()**2
        imags = (bc - ad).sum()**2
        norm = (reals + imags) / (psi1.shape[1]**2)
        return norm
    
    
    @staticmethod
    def c_to_r_mat(M):
        # complex to real isomorphism for matrix
        return np.asarray(np.bmat([[M.real, -M.imag],[M.imag, M.real]]))

    @staticmethod
    def c_to_r_vec(V):
        # complex to real isomorphism for vector
        new_v = []
        new_v.append(V.real)
        new_v.append(V.imag)
        return np.reshape(new_v, [2 * len(V)])
        
    @staticmethod
    def random_initialize_u(n_step, n_Hs):
        initial_mean = 0
        #initial_stddev =  (1. / np.sqrt(n_step))
        initial_stddev = 2
        u = np.random.normal(initial_mean, initial_stddev, [n_step ,n_Hs])
        
        return u
    


    #====================================================================
    def ibm_single_plus(self, duration):
        """synthesizing the plus state on the IBM machine
        """
        self.logger.write_text("ibm_single_plus========")

        n_step = self.n_step
        T = duration * 0.22
        self.dt = T / self.n_step
        w0 = 5236376147.050786 * 2 * np.pi * 1e-9
        eps0 = 32901013497.991684 * 1e-9
        Omega0 = 955111374.7779446 * 1e-9
        Delta0 = eps0 - w0

        I = np.array([[1, 0], 
                    [0, 1]])
        X = np.array([[0, 1], 
                    [1, 0]])
        Y = (0+1j) * np.array([[0, -1], 
                            [1, 0]])
        Z = np.array([[1, 0], 
                    [0, -1]])
        
        # apply RWA
        H0 = 0.5 * Delta0 * (I-Z)
        Hs = [0.5 * Omega0 * X, 0.5 * Omega0 * Y]
        
        Udag0 = np.array([[1, 0], 
                    [0, np.exp(-1.j * w0 * T)]])

        g = np.array([1., 0.])
        e = np.array([0., 1.])
        plus = 1/np.sqrt(2) * (g+e)
        
        psi_in = [g]
        psi_out = [Udag0 @ plus]

        target_states = [self.c_to_r_vec(v) for v in psi_out]
        initial_states = [self.c_to_r_vec(v) for v in psi_in]

        init_u = self.random_initialize_u(n_step, len(Hs))
        final_u = self.train_fidelity_ibm1(init_u, H0, Hs, initial_states, target_states)
    
    
    def ibm_X(self, duration):
        """synthesizing the plus state on the IBM machine
        """
        self.logger.write_text("ibm_X========")

        n_step = self.n_step
        T = duration * 0.22
        self.dt = T / self.n_step
        w0 = 5236376147.050786 * 2 * np.pi * 1e-9
        eps0 = 32901013497.991684 * 1e-9
        Omega0 = 955111374.7779446 * 1e-9
        Delta0 = eps0 - w0

        I = np.array([[1, 0], 
                    [0, 1]])
        X = np.array([[0, 1], 
                    [1, 0]])
        Y = (0+1j) * np.array([[0, -1], 
                            [1, 0]])
        Z = np.array([[1, 0], 
                    [0, -1]])
        
        # apply RWA
        H0 = 0.5 * Delta0 * (I-Z)
        Hs = [0.5 * Omega0 * X, 0.5 * Omega0 * Y]
        
        Udag0 = np.array([[1, 0], 
                    [0, np.exp(-1.j * w0 * T)]])

        g = np.array([1., 0.])
        e = np.array([0., 1.])
        plus = 1/np.sqrt(2) * (g+e)
        
        psi_in = [g, e, plus]
        psi_out = [Udag0 @ e, Udag0 @ g, Udag0 @ plus]

        target_states = [self.c_to_r_vec(v) for v in psi_out]
        initial_states = [self.c_to_r_vec(v) for v in psi_in]

        init_u = self.random_initialize_u(n_step, len(Hs))
        final_u = self.train_fidelity_ibm1(init_u, H0, Hs, initial_states, target_states)
    
        
        
    #====================================================================
    def ibm_bell_state(self, duration):
        """synthesizing the X gate on the IBM machine
        """
        self.logger.write_text("ibm_bell_state ========")

        n_step = self.n_step
        T = duration * 0.22
        self.dt = T / self.n_step
        w0 = 5236376147.050786 * 2 * np.pi * 1e-9
        w1 = 5014084426.228487 * 2 * np.pi * 1e-9
        self.delta = w1 - w0
        eps0 = 32901013497.991684 * 1e-9
        eps1 = 31504959831.439907 * 1e-9
        Omega0 = 955111374.7779446 * 1e-9
        Omega1 = 987150040.8532522 * 1e-9
        Delta0 = eps0 - w0
        Delta1 = eps1 - w1
        j01 = 12286377.631357463 * 1e-9

        I = np.array([[1, 0], 
                    [0, 1]])
        X = np.array([[0, 1], 
                    [1, 0]])
        Y = (0+1j) * np.array([[0, -1], 
                            [1, 0]])
        Z = np.array([[1, 0], 
                    [0, -1]])
        
        Hint_re = np.array([[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,0]])
        Hint_im = 1.j* np.array([[0,0,0,0],[0,0,-1,0],[0,1,0,0],[0,0,0,0]])
        
        #U_to_learn = X
        #print(U_to_learn)

        # apply RWA
        Hsys_1q = 0.5 * (I-Z)
        Id = np.eye(2)
        H0 = [Delta0 * kron(Hsys_1q,Id)+Delta1 * kron(Id, Hsys_1q), 
              j01*Hint_re, j01*Hint_im]
        Hs = [0.5 * Omega0 * kron(X,Id), 0.5 * Omega0 * kron(Y,Id),
              0.5 * Omega1 * kron(Id,X), 0.5 * Omega1 * kron(Id,Y)]
        
        Udag0 = np.array([[1, 0], 
                    [0, np.exp(-1.j * w0 * T)]])
        Udag1 = np.array([[1, 0], 
                    [0, np.exp(-1.j * w1 * T)]])
        Udag = kron(Udag0, Udag1)

        g = np.array([1., 0.])
        e = np.array([0., 1.])
        plus = 1/np.sqrt(2) * (g+e)
        state_in = kron(g,g)
        state_out = 1/np.sqrt(2) * (kron(g,g)+kron(e,e))
        psi_in = [state_in]
        psi_out = [Udag @ state_out]

        target_states = [self.c_to_r_vec(v) for v in psi_out]
        initial_states = [self.c_to_r_vec(v) for v in psi_in]

        init_u = self.random_initialize_u(n_step, 2*len(Hs))  
        final_u = self.train_fidelity_ibm2(init_u, H0, Hs, initial_states, target_states)
    
    
    
    def ibm_CNOT(self, duration):
        """synthesizing the X gate on the IBM machine
        """
        self.logger.write_text("ibm_CNOT ========")

        n_step = self.n_step
        T = duration * 0.22
        self.dt = T / self.n_step
        w0 = 5236376147.050786 * 2 * np.pi * 1e-9
        w1 = 5014084426.228487 * 2 * np.pi * 1e-9
        self.delta = w1 - w0
        eps0 = 32901013497.991684 * 1e-9
        eps1 = 31504959831.439907 * 1e-9
        Omega0 = 955111374.7779446 * 1e-9
        Omega1 = 987150040.8532522 * 1e-9
        Delta0 = eps0 - w0
        Delta1 = eps1 - w1
        j01 = 12286377.631357463 * 1e-9

        I = np.array([[1, 0], 
                    [0, 1]])
        X = np.array([[0, 1], 
                    [1, 0]])
        Y = (0+1j) * np.array([[0, -1], 
                            [1, 0]])
        Z = np.array([[1, 0], 
                    [0, -1]])
        
        Hint_re = np.array([[0,0,0,0],[0,0,1,0],[0,1,0,0],[0,0,0,0]])
        Hint_im = 1.j* np.array([[0,0,0,0],[0,0,-1,0],[0,1,0,0],[0,0,0,0]])
        
        #U_to_learn = X
        #print(U_to_learn)

        # apply RWA
        Hsys_1q = 0.5 * (I-Z)
        Id = np.eye(2)
        H0 = [Delta0 * kron(Hsys_1q,Id)+Delta1 * kron(Id, Hsys_1q), 
              j01*Hint_re, j01*Hint_im]
        Hs = [0.5 * Omega0 * kron(X,Id), 0.5 * Omega0 * kron(Y,Id),
              0.5 * Omega1 * kron(Id,X), 0.5 * Omega1 * kron(Id,Y)]
        
        Udag0 = np.array([[1, 0], 
                    [0, np.exp(-1.j * w0 * T)]])
        Udag1 = np.array([[1, 0], 
                    [0, np.exp(-1.j * w1 * T)]])
        Udag = kron(Udag0, Udag1)

        g = np.array([1., 0.])
        e = np.array([0., 1.])
        plus = 1/np.sqrt(2) * (g+e)
        state_in1 = kron(g,g)
        state_in2 = kron(g,e)
        state_in3 = kron(e,g)
        state_in4 = kron(e,e)
        state_in5 = kron(plus,plus)
        psi_in = [state_in1, state_in2, state_in3, state_in4, state_in5]
        psi_out = [Udag @ state_in1, Udag @ state_in2, Udag @ state_in4, Udag @ state_in3, Udag @ state_in5]

        target_states = [self.c_to_r_vec(v) for v in psi_out]
        initial_states = [self.c_to_r_vec(v) for v in psi_in]

        init_u = self.random_initialize_u(n_step, 2*len(Hs))
        final_u = self.train_fidelity_ibm2(init_u, H0, Hs, initial_states, target_states)

        #plt.loglog(self.losses_energy)
        #plt.savefig("grape.png")

In [ ]:
if __name__ == '__main__':
    grape = Grape(taylor_terms=20, n_step=100,lr=1e-2, n_epoch=1200)
    # grape.ibm_single_plus(duration=20)
    # grape.ibm_X(duration=40)
    # grape.ibm_bell_state(duration=500)
    grape.ibm_CNOT(duration=1200)

logs are written to ./logs/text/20220120_145329.txt
arguments ========
taylor_terms: 20
n_step: 100
n_epoch: 1200
lr: 0.01
ibm_CNOT ========
!!!! train_fidelity ========
epoch: 0000, loss: 0.9878234659247654, loss_fid: 0.9878234659247654
epoch: 0001, loss: 0.7930937290810443, loss_fid: 0.7930937290810443
epoch: 0002, loss: 0.5605988064723859, loss_fid: 0.5605988064723859
epoch: 0003, loss: 0.4676872166486058, loss_fid: 0.4676872166486058
epoch: 0004, loss: 0.5635598748983952, loss_fid: 0.5635598748983952
epoch: 0005, loss: 0.5854343075050650, loss_fid: 0.5854343075050650
epoch: 0006, loss: 0.5279064149060311, loss_fid: 0.5279064149060311
epoch: 0007, loss: 0.4543136360970041, loss_fid: 0.4543136360970041
epoch: 0008, loss: 0.4122841383573788, loss_fid: 0.4122841383573788
epoch: 0009, loss: 0.4326790415373634, loss_fid: 0.4326790415373634
epoch: 0010, loss: 0.4617616503662105, loss_fid: 0.4617616503662105
epoch: 0011, loss: 0.4559851357981310, loss_fid: 0.4559851357981310
epoch: 0012, l

epoch: 0119, loss: 0.1238825869738608, loss_fid: 0.1238825869738608
epoch: 0120, loss: 0.1229136525162804, loss_fid: 0.1229136525162804
epoch: 0121, loss: 0.1219795212232014, loss_fid: 0.1219795212232014
epoch: 0122, loss: 0.1211197231602767, loss_fid: 0.1211197231602767
epoch: 0123, loss: 0.1204511403562870, loss_fid: 0.1204511403562870
epoch: 0124, loss: 0.1202900264870798, loss_fid: 0.1202900264870798
epoch: 0125, loss: 0.1210556274710797, loss_fid: 0.1210556274710797
epoch: 0126, loss: 0.1219910302324664, loss_fid: 0.1219910302324664
epoch: 0127, loss: 0.1198512364540365, loss_fid: 0.1198512364540365
epoch: 0128, loss: 0.1161046577766897, loss_fid: 0.1161046577766897
epoch: 0129, loss: 0.1159742276564185, loss_fid: 0.1159742276564185
epoch: 0130, loss: 0.1168322026442786, loss_fid: 0.1168322026442786
epoch: 0131, loss: 0.1145656060594287, loss_fid: 0.1145656060594287
epoch: 0132, loss: 0.1130240870900456, loss_fid: 0.1130240870900456
epoch: 0133, loss: 0.1135528388064014, loss_fid:

epoch: 0241, loss: 0.0641890528479737, loss_fid: 0.0641890528479737
epoch: 0242, loss: 0.0639986326049491, loss_fid: 0.0639986326049491
epoch: 0243, loss: 0.0637685502797470, loss_fid: 0.0637685502797470
epoch: 0244, loss: 0.0634416860970544, loss_fid: 0.0634416860970544
epoch: 0245, loss: 0.0631012913880723, loss_fid: 0.0631012913880723
epoch: 0246, loss: 0.0628273657069759, loss_fid: 0.0628273657069759
epoch: 0247, loss: 0.0625973550498710, loss_fid: 0.0625973550498710
epoch: 0248, loss: 0.0623484894694961, loss_fid: 0.0623484894694961
epoch: 0249, loss: 0.0620667554022519, loss_fid: 0.0620667554022519
epoch: 0250, loss: 0.0617826794777724, loss_fid: 0.0617826794777724
epoch: 0251, loss: 0.0615145278222177, loss_fid: 0.0615145278222177
epoch: 0252, loss: 0.0612526998518298, loss_fid: 0.0612526998518298
epoch: 0253, loss: 0.0609887866262631, loss_fid: 0.0609887866262631
epoch: 0254, loss: 0.0607296570971830, loss_fid: 0.0607296570971830
epoch: 0255, loss: 0.0604809905812523, loss_fid:

epoch: 0363, loss: 0.0399507879200433, loss_fid: 0.0399507879200433
epoch: 0364, loss: 0.0398154176320098, loss_fid: 0.0398154176320098
epoch: 0365, loss: 0.0396842493690643, loss_fid: 0.0396842493690643
epoch: 0366, loss: 0.0395594328050493, loss_fid: 0.0395594328050493
epoch: 0367, loss: 0.0394444429851727, loss_fid: 0.0394444429851727
epoch: 0368, loss: 0.0393455224224090, loss_fid: 0.0393455224224090
epoch: 0369, loss: 0.0392726728494480, loss_fid: 0.0392726728494480
epoch: 0370, loss: 0.0392437297783594, loss_fid: 0.0392437297783594
epoch: 0371, loss: 0.0392852230585572, loss_fid: 0.0392852230585572
epoch: 0372, loss: 0.0394407272724979, loss_fid: 0.0394407272724979
epoch: 0373, loss: 0.0397552362930473, loss_fid: 0.0397552362930473
epoch: 0374, loss: 0.0402697141456244, loss_fid: 0.0402697141456244
epoch: 0375, loss: 0.0409001829134344, loss_fid: 0.0409001829134344
epoch: 0376, loss: 0.0414157830676199, loss_fid: 0.0414157830676199
epoch: 0377, loss: 0.0413248837915655, loss_fid:

epoch: 0485, loss: 0.0288451158681821, loss_fid: 0.0288451158681821
epoch: 0486, loss: 0.0291108964254491, loss_fid: 0.0291108964254491
epoch: 0487, loss: 0.0292525242583592, loss_fid: 0.0292525242583592
epoch: 0488, loss: 0.0291946293842824, loss_fid: 0.0291946293842824
epoch: 0489, loss: 0.0288189062103459, loss_fid: 0.0288189062103459
epoch: 0490, loss: 0.0281923911237943, loss_fid: 0.0281923911237943
epoch: 0491, loss: 0.0274754257041329, loss_fid: 0.0274754257041329
epoch: 0492, loss: 0.0269198282428009, loss_fid: 0.0269198282428009
epoch: 0493, loss: 0.0266721452224842, loss_fid: 0.0266721452224842
epoch: 0494, loss: 0.0267197551356035, loss_fid: 0.0267197551356035
epoch: 0495, loss: 0.0269282642121864, loss_fid: 0.0269282642121864
epoch: 0496, loss: 0.0271285547365955, loss_fid: 0.0271285547365955
epoch: 0497, loss: 0.0271947564529960, loss_fid: 0.0271947564529960
epoch: 0498, loss: 0.0270669969718266, loss_fid: 0.0270669969718266
epoch: 0499, loss: 0.0267883623384267, loss_fid:

epoch: 0607, loss: 0.0199719246541129, loss_fid: 0.0199719246541129
epoch: 0608, loss: 0.0198920554848285, loss_fid: 0.0198920554848285
epoch: 0609, loss: 0.0198194984715060, loss_fid: 0.0198194984715060
epoch: 0610, loss: 0.0197555651644116, loss_fid: 0.0197555651644116
epoch: 0611, loss: 0.0196992310876763, loss_fid: 0.0196992310876763
epoch: 0612, loss: 0.0196484738867337, loss_fid: 0.0196484738867337
epoch: 0613, loss: 0.0196011337678302, loss_fid: 0.0196011337678302
epoch: 0614, loss: 0.0195555850841426, loss_fid: 0.0195555850841426
epoch: 0615, loss: 0.0195110012580224, loss_fid: 0.0195110012580224
epoch: 0616, loss: 0.0194672734053231, loss_fid: 0.0194672734053231
epoch: 0617, loss: 0.0194247776680163, loss_fid: 0.0194247776680163
epoch: 0618, loss: 0.0193842168026768, loss_fid: 0.0193842168026768
epoch: 0619, loss: 0.0193461905842544, loss_fid: 0.0193461905842544
epoch: 0620, loss: 0.0193112843411691, loss_fid: 0.0193112843411691
epoch: 0621, loss: 0.0192799284355774, loss_fid:

epoch: 0729, loss: 0.0155214329643867, loss_fid: 0.0155214329643867
epoch: 0730, loss: 0.0153129724291354, loss_fid: 0.0153129724291354
epoch: 0731, loss: 0.0151800909933775, loss_fid: 0.0151800909933775
epoch: 0732, loss: 0.0151560530058552, loss_fid: 0.0151560530058552
epoch: 0733, loss: 0.0152096113425984, loss_fid: 0.0152096113425984
epoch: 0734, loss: 0.0152734451697757, loss_fid: 0.0152734451697757
epoch: 0735, loss: 0.0152855794710319, loss_fid: 0.0152855794710319
epoch: 0736, loss: 0.0152222637313093, loss_fid: 0.0152222637313093
epoch: 0737, loss: 0.0151052067830025, loss_fid: 0.0151052067830025
epoch: 0738, loss: 0.0149822560914968, loss_fid: 0.0149822560914968
epoch: 0739, loss: 0.0148966343189588, loss_fid: 0.0148966343189588
epoch: 0740, loss: 0.0148642728777543, loss_fid: 0.0148642728777543
epoch: 0741, loss: 0.0148715505873553, loss_fid: 0.0148715505873553
epoch: 0742, loss: 0.0148890100733761, loss_fid: 0.0148890100733761
epoch: 0743, loss: 0.0148895739349273, loss_fid: